# Toronto postal codes and neighbourhoods retrieval

The present workbook aims to scrape the postal codes and neighbourhoods of Toronto from the following <a href="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">link from Wikipedia</a>.


## I. Libraries preparations

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
!conda install -c conda-forge bs4 --yes 
from bs4 import BeautifulSoup
!conda install -c conda-forge lxml --yes 
import lxml
!conda install -c conda-forge requests --yes 
import requests
print("Done!")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Done!


## II. Web-data scrapping

In [6]:
#----------------
# Web data extraction
#----------------
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
soup = BeautifulSoup(requests.get(url).content, "html.parser")
#soup = BeautifulSoup(requests.get(url).content, "lxml")
#print(soup.prettify())


In [3]:
#----------------
# Reading table
#----------------
table=soup.find('table', class_="wikitable sortable")
#print(table.prettify())

#------------------------------------------------------------------------------
# Scanning through the table and extracting row info into a list (table_data)
#------------------------------------------------------------------------------
table_rows=table.find_all('tr')
table_data=[]
for tr in table_rows:
    td = tr.find_all('td')
    entry = [i.text for i in td]
    table_data.append(entry)
    #print(entry)

#------------------------------------------------
# Creating a df based on row info in table_tada
#------------------------------------------------
table_df = pd.DataFrame(table_data, columns=["PostalCode", "Borough", "Neighbourhood"])
print(table_df.shape)
table_df.head()

(181, 3)


,PostalCode,Borough,Neighbourhood
0,None,None,None
1,M1A\n,Not assigned\n,\n
2,M2A\n,Not assigned\n,\n
3,M3A\n,North York\n,Parkwoods\n
4,M4A\n,North York\n,Victoria Village\n


## III. Data manipulation and preparation

in this chapter, the data collected and stored in the dataframe will be manipulated to meet the criteria from the assignment being it:
* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.
* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [4]:
#----------------------------------
# DF cleaning and re-organisation
#----------------------------------
#Removing the \n 
table_df.replace(to_replace ='\n', value = '', regex = True, inplace=True)
table_df.head()

#Discarding not assigned boroughs (Not assigned / None)
table_df.drop(table_df.loc[ (table_df['Borough'] == 'Not assigned')].index, inplace=True)
table_df.dropna(inplace=True)

#Changing separators from "/" to ","
table_df['Neighbourhood'].replace(to_replace =' /', value = ',', regex = True, inplace=True)

#Updating Bourhoods with missing Neighbourhoods
table_df['Neighbourhood']= np.where(table_df['Neighbourhood']== 'Not assigned', table_df['Borough'], table_df['Neighbourhood'])

#Resetting the index
table_df.reset_index(inplace=True, drop=True)
table_df.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


## IV. Results

This setion will show the result of the previous analyses.

In [5]:
table_df.shape

(103, 3)